## Kaggle Dataset의 일부를 이용한 개, 고양이 구분ㅡ

### Dog Image 1,111개, Cat Image 1111개 총 2,222개

In [1]:
import numpy as np
import tensorflow as tf
import os, cv2, random
import matplotlib.pyplot as plt
from glob import glob
from tensorflow import keras
from keras.preprocessing import image
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
%matplotlib inline

Using TensorFlow backend.


In [2]:
seed=2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [3]:
path='../dataset/cat_and_dog/train/'

In [4]:
## used for resize and in our model
ROW, COL=96, 96
dogs, cats=[],[]
y_dogs, y_cats=[],[]

In [5]:
dog_path=os.path.join(path, 'dog.*')
cat_path=os.path.join(path, 'cat.*')
len(glob(dog_path))

12500

In [6]:
## Load some our dog image
for dog_img in glob(dog_path):
    dog=cv2.imread(dog_img)
    dog=cv2.cvtColor(dog,cv2.COLOR_BGR2GRAY)
    dog=cv2.resize(dog, (ROW, COL))
    dog=image.img_to_array(dog)
    dogs.append(dog)

In [7]:
## Load some our dog image
for cat_img in glob(cat_path):
    cat=cv2.imread(cat_img)
    cat=cv2.cvtColor(cat,cv2.COLOR_BGR2GRAY)
    cat=cv2.resize(cat, (ROW, COL))
    cat=image.img_to_array(cat)
    cats.append(cat)

In [8]:
classes=['dog','cat']

In [9]:
## just change the labels for 0 and 1
y_dogs=[1 for item in enumerate(dogs)]
y_cats=[0 for item in enumerate(cats)]

In [10]:
## converting everything to Numpy array to fit in our model
## them creating a X and target file like we used to see
## in Machine and Deep Learning models
dogs = np.asarray(dogs).astype('float32') / 255
cats = np.asarray(cats).astype('float32') / 255
y_dogs = np.asarray(y_dogs).astype('int32')
y_cats = np.asarray(y_cats).astype('int32')

In [11]:
x = np.concatenate((dogs,cats), axis=0)
y = np.concatenate((y_dogs, y_cats), axis=0)

In [12]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.2,random_state=seed)

In [13]:
x_test.shape

(5000, 96, 96, 1)

In [14]:
## Here is our model as a CNN
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(ROW, COL, 1), activation='relu'),
    Conv2D(32, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(.5),
    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 24, 24, 64)       

In [19]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [20]:
## to save checkpoint to use later
modelpath = "model/dogs_vs_cats-cnn.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

In [21]:
model.fit(x_train, y_train, batch_size=100, epochs=200, validation_split=0.2,shuffle=True,
          callbacks=[checkpointer, early_stopping_callback])

Train on 16000 samples, validate on 4000 samples
Epoch 1/200
16000/16000 [==============================] - 445s 28ms/step - loss: 0.6912 - accuracy: 0.5394 - val_loss: 0.6748 - val_accuracy: 0.5918

Epoch 00001: val_loss improved from inf to 0.67477, saving model to model/dogs_vs_cats-cnn.hdf5
Epoch 2/200
16000/16000 [==============================] - 472s 29ms/step - loss: 0.6625 - accuracy: 0.6029 - val_loss: 0.6134 - val_accuracy: 0.6488

Epoch 00002: val_loss improved from 0.67477 to 0.61338, saving model to model/dogs_vs_cats-cnn.hdf5
Epoch 3/200
16000/16000 [==============================] - 471s 29ms/step - loss: 0.5753 - accuracy: 0.7041 - val_loss: 0.5505 - val_accuracy: 0.7237

Epoch 00003: val_loss improved from 0.61338 to 0.55046, saving model to model/dogs_vs_cats-cnn.hdf5
Epoch 4/200
16000/16000 [==============================] - 353s 22ms/step - loss: 0.5071 - accuracy: 0.7497 - val_loss: 0.4971 - val_accuracy: 0.7598

Epoch 00004: val_loss improved from 0.55046 to 0.49

KeyboardInterrupt: 

In [ ]:
del model
model = load_model('model/dogs_vs_cats.hdf5')

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=2)
print('MODEL ACCURACY: %.5f' % scores[1])